In [1]:
import pandas as pd
import numpy as np
# import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn import preprocessing

seed = 42

FILENAME = "train_dataset.csv"

#Prepare train data
df1 = pd.read_csv(FILENAME, encoding='ISO-8859-1', sep=",", low_memory=False)
# print("EX1) #Righe: " + str(df1.shape[0])+ " #Colonne: "+str(df1.shape[1]))

# print(df1.nunique())
# print(df1.isna().sum())

# print(df1.shape)
df1 = df1.dropna()
# print(df1.shape)

In [2]:
y = df1["type"]
X = df1.drop(columns=["type", "label"])

err_arr = X.loc[X["src_bytes"] == "0.0.0.0"]
X = X.drop(index=err_arr.index)
y = y.drop(index=err_arr.index)
X.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

X = X.to_numpy()
y = y.to_numpy()

for feature in ["dns_AA","dns_RD","dns_RA","dns_rejected","ssl_version","ssl_cipher","ssl_resumed","ssl_established","ssl_subject","ssl_issuer","http_trans_depth","http_method","http_uri","http_referrer","http_version","http_request_body_len","http_response_body_len","http_status_code","http_user_agent","http_orig_mime_types","http_resp_mime_types","weird_name","weird_addl","weird_notice"]:
    # print(f"Feature: {feature}")    
    feature_index = np.where(df1.columns == feature)[0][0]
    elements, counts = np.unique(X[:, feature_index], return_counts=True)

    # for element, count in zip(elements, counts):
    #     print(f"    Element: {element}, Count: {count}")

In [3]:
# oe = preprocessing.OrdinalEncoder()
# oe.fit(X)
# X = oe.transform(X)

# le = preprocessing.LabelEncoder()
# le.fit(y)
# y = le.transform(y)

In [7]:
## FOR MINMAX SCALING
# oe = preprocessing.OrdinalEncoder()
# oe.fit(X)
# X = oe.transform(X)

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Check if columns are numerical, if not use OrdinalEncoder
for col in df1.columns:
    if df1[col].dtype == 'object':
        col_index = df1.columns.get_loc(col)
        if col_index < X.shape[1]:  # Ensure the column index is within bounds
            oe = preprocessing.OrdinalEncoder()
            X[:, col_index] = oe.fit_transform(X[:, col_index].reshape(-1, 1)).flatten()

print(X)

[[1554350584 296.0 53972 ... 0.0 0.0 0.0]
 [1556087112 302.0 37513 ... 0.0 0.0 0.0]
 [1556140763 296.0 2077 ... 0.0 0.0 0.0]
 ...
 [1556547123 311.0 54730 ... 0.0 0.0 0.0]
 [1556547464 314.0 59846 ... 0.0 0.0 0.0]
 [1556547114 311.0 56698 ... 0.0 0.0 0.0]]


In [4]:
# variances = np.var(X, axis=0)
# features = df1.columns
# for i, variance in enumerate(variances):
#     print(f"{features[i]} \t\t: Variance = {variance}")

ts 		: Variance = 1783833117.1326857
src_ip 		: Variance = 284.11687981463507
src_port 		: Variance = 226183441.1094727
dst_ip 		: Variance = 18069.481632403946
dst_port 		: Variance = 112729606.3787938
proto 		: Variance = 0.07609999553869298
service 		: Variance = 17.47203143096537
duration 		: Variance = 3668014251.625104
src_bytes 		: Variance = 1243377.3513259336
dst_bytes 		: Variance = 275244.427664733
conn_state 		: Variance = 12.47918490190582
missed_bytes 		: Variance = 2172.8186935295867
src_pkts 		: Variance = 138.1312245942286
src_ip_bytes 		: Variance = 369389.06178173196
dst_pkts 		: Variance = 8.776803127539445
dst_ip_bytes 		: Variance = 585861.2144607559
dns_query 		: Variance = 208822.20217437387
dns_qclass 		: Variance = 0.06832388472089732
dns_qtype 		: Variance = 0.26119829202000905
dns_rcode 		: Variance = 0.12338129665094301
dns_AA 		: Variance = 0.07302113656078127
dns_RD 		: Variance = 0.259392226367049
dns_RA 		: Variance = 0.16315153361398274
dns_rejected 		

In [8]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(0.01)
X_mod = selector.fit_transform(X)

print(f"Original shape: {X.shape}")
print(f"Modified shape: {X_mod.shape}")

Original shape: (616983, 44)
Modified shape: (616983, 30)


In [9]:
train_idx, test_idx = train_test_split(np.arange(X_mod.shape[0]), test_size=0.2, stratify=y, random_state=seed)
test_idx, val_idx = train_test_split(test_idx, test_size=0.5, stratify=y[test_idx], random_state=seed)

y_test = y[test_idx]
y_train = y[train_idx]
y_val = y[val_idx]

X_test_mod = X_mod[test_idx,:]
X_train_mod = X_mod[train_idx,:]
X_val_mod = X_mod[val_idx,:]

X_test = X[test_idx,:]
X_train = X[train_idx,:]
X_val = X[val_idx,:]

print(X_train.shape, X_test.shape, X_val.shape)
print(X_train_mod.shape, X_test_mod.shape, X_val_mod.shape)
print(len(y_train), len(y_test), len(y_val))

(493586, 44) (61698, 44) (61699, 44)
(493586, 30) (61698, 30) (61699, 30)
493586 61698 61699


In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import decomposition

## Scaling (treshold)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_mod)
X_train_mod = scaler.transform(X_train_mod)
X_test_mod = scaler.transform(X_test_mod)
X_val_mod = scaler.transform(X_val_mod)

## Scaling (no treshold)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

print(X_train.shape, X_test.shape, X_val.shape)
print(X_train_mod.shape, X_test_mod.shape, X_val_mod.shape)


## PCA (treshold)
pca = decomposition.PCA()
pca.fit(X_train_mod)
X_train_mod_pca = pca.transform(X_train_mod)
X_test_mod_pca = pca.transform(X_test_mod)
X_val_mod_pca = pca.transform(X_val_mod)

## PCA (no treshold)
pca = decomposition.PCA()
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_val_pca = pca.transform(X_val)

print(X_train_pca.shape, X_test_pca.shape, X_val_pca.shape)
print(X_train_mod_pca.shape, X_test_mod_pca.shape, X_val_mod_pca.shape)


## LDA (treshold)
lda = LinearDiscriminantAnalysis(n_components=9)
lda.fit(X_train_mod, y_train)
X_train_mod_lda = lda.transform(X_train_mod)
X_test_mod_lda = lda.transform(X_test_mod)
X_val_mod_lda = lda.transform(X_val_mod)

## LDA (no treshold)
lda = LinearDiscriminantAnalysis(n_components=9)
lda.fit(X_train, y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)
X_val_lda = lda.transform(X_val)

print(X_train_lda.shape, X_test_lda.shape, X_val_lda.shape)
print(X_train_mod_lda.shape, X_test_mod_lda.shape, X_val_mod_lda.shape)

(493586, 44) (61698, 44) (61699, 44)
(493586, 28) (61698, 28) (61699, 28)
(493586, 44) (61698, 44) (61699, 44)
(493586, 28) (61698, 28) (61699, 28)
(493586, 9) (61698, 9) (61699, 9)
(493586, 9) (61698, 9) (61699, 9)


In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

scaler_mod = MinMaxScaler()
X_train_mod = scaler.fit_transform(X_train_mod)
X_test_mod = scaler.transform(X_test_mod)
X_val_mod = scaler.transform(X_val_mod)

---

In [8]:
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, balanced_accuracy_score
from sklearn import svm

---

### PCA, threshold

In [9]:
## PCA, treshold
clf_mod_pca = svm.LinearSVC(random_state=seed)
clf_mod_pca.fit(X_train_mod_pca, y_train)
y_mod_pca_pred = clf_mod_pca.predict(X_test_mod_pca)

print(accuracy_score(y_test, y_mod_pca_pred))

0.9485234529482317


In [10]:
clf = clf_mod_pca
x = X_val_mod_pca
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     1     0     0     0     0     0     0     0     0]
 [    0 17651   129   111     0   296    53     0     0     9]
 [    0    25  5342     6     0    70     0     0     5     0]
 [    0    97    16  1219     0    13     4     0     9     0]
 [    0     9     1     0    34     2     1     0     0    15]
 [   23   104   168    12    13  1233    21    55   417    20]
 [    0  1103     1     0     0    16  4021     0     0    15]
 [    5     4     0     0     0     7     1    71     0     8]
 [    0    12   139    36     0    31     2     0 21201     0]
 [    0    76     0     0     0     7   108     1     0  6135]]
0.9468873077359439
0.9453073456296222
0.9462369773053101
0.9468873077359439
0.8518055415975313
0.865204355626517
0.846878589763229
0.846878589763229


In [11]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1516
           1       0.93      0.97      0.95     18249
           2       0.92      0.98      0.95      5448
           3       0.88      0.90      0.89      1358
           4       0.72      0.55      0.62        62
           5       0.74      0.60      0.66      2066
           6       0.95      0.78      0.86      5156
           7       0.56      0.74      0.64        96
           8       0.98      0.99      0.98     21421
           9       0.99      0.97      0.98      6327

    accuracy                           0.95     61699
   macro avg       0.87      0.85      0.85     61699
weighted avg       0.95      0.95      0.95     61699



---

### PCA, no threshold

In [12]:
## PCA, no treshold
clf_pca = svm.LinearSVC(random_state=seed)
clf_pca.fit(X_train_pca, y_train)
y_pca_pred = clf_pca.predict(X_test_pca)

print(accuracy_score(y_test, y_pca_pred))

0.9486044928522804


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [13]:
clf = clf_pca
x = X_val_pca
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     1     0     0     0     0     0     0     0     0]
 [    0 17649   129   110     0   297    55     0     0     9]
 [    0    25  5342     6     0    70     0     0     5     0]
 [    0    97    16  1220     0    12     4     0     9     0]
 [    0     9     1     0    34     2     1     0     0    15]
 [   24   104   168    12    13  1233    21    53   418    20]
 [    0  1101     1     0     0    16  4023     0     0    15]
 [    5     4     0     0     0     7     1    71     0     8]
 [    1    12   138    36     0    31     2     0 21201     0]
 [    0    76     0     0     0     7   106     1     0  6137]]
0.9469359308902899
0.9453513294773028
0.9462623864201881
0.9469359308902899
0.8524362644010278
0.866066772626581
0.8470116682785325
0.8470116682785325


In [14]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1516
           1       0.93      0.97      0.95     18249
           2       0.92      0.98      0.95      5448
           3       0.88      0.90      0.89      1358
           4       0.72      0.55      0.62        62
           5       0.74      0.60      0.66      2066
           6       0.95      0.78      0.86      5156
           7       0.57      0.74      0.64        96
           8       0.98      0.99      0.98     21421
           9       0.99      0.97      0.98      6327

    accuracy                           0.95     61699
   macro avg       0.87      0.85      0.85     61699
weighted avg       0.95      0.95      0.95     61699



---

### LDA, threshold

In [15]:
## LDA, treshold
clf_mod_lda = svm.LinearSVC(random_state=seed)
clf_mod_lda.fit(X_train_mod_lda, y_train)
y_mod_lda_pred = clf_mod_lda.predict(X_test_mod_lda)

print(accuracy_score(y_test, y_mod_lda_pred))

0.9252812084670492


In [16]:
clf = clf_mod_lda
x = X_val_mod_lda
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     0     0     0     0     0     1     0     0     0]
 [    0 17606   145    68     0   290   137     0     0     3]
 [    0    23  5287    33     0    70     0     0    35     0]
 [    0   125    30  1163     0     4    14     0    22     0]
 [    1    10     1     0    19     0     3     0     0    28]
 [   58    91   201    64     9   995   123    55   388    82]
 [    0  1741     7     1     0    97  3228     0     0    82]
 [   43     0     0     0     0     8    34     1     0    10]
 [    0    18   185    46     0    27     0     0 21145     0]
 [    1   130     0     0     0    19   120     0     0  6057]]
0.9240992560657385
0.9195278108733755
0.9207860797473512
0.9240992560657385
0.7349796573159308
0.7760695624659854
0.7159943075106613
0.7159943075106613


In [17]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1516
           1       0.89      0.96      0.93     18249
           2       0.90      0.97      0.94      5448
           3       0.85      0.86      0.85      1358
           4       0.68      0.31      0.42        62
           5       0.66      0.48      0.56      2066
           6       0.88      0.63      0.73      5156
           7       0.02      0.01      0.01        96
           8       0.98      0.99      0.98     21421
           9       0.97      0.96      0.96      6327

    accuracy                           0.92     61699
   macro avg       0.78      0.72      0.73     61699
weighted avg       0.92      0.92      0.92     61699



---

### LDA, no threshold

In [18]:
## LDA, no treshold
clf_lda = svm.LinearSVC(random_state=seed)
clf_lda.fit(X_train_lda, y_train)
y_lda_pred = clf_lda.predict(X_test_lda)

print(accuracy_score(y_test, y_lda_pred))

0.9254757042367662


In [19]:
clf = clf_lda
x = X_val_lda
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     0     0     0     0     0     1     0     0     0]
 [    0 17605   144    68     0   290   139     0     0     3]
 [    0    23  5286    34     0    70     0     0    35     0]
 [    0   125    30  1163     0     4    14     0    22     0]
 [    0    11     1     0    19     0     3     0     0    28]
 [   58    89   200    63     9   999   123    54   388    83]
 [    0  1737     7     1     0    96  3233     0     0    82]
 [   44     0     0     0     0     8    33     1     0    10]
 [    0    18   185    47     0    27     0     0 21144     0]
 [    1   131     0     0     0    19   117     0     0  6059]]
0.9242289178106614
0.9196819299496829
0.9209340446538379
0.9242289178106614
0.7352431468408593
0.7762707694766096
0.7162879998813347
0.7162879998813347


In [20]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1516
           1       0.89      0.96      0.93     18249
           2       0.90      0.97      0.94      5448
           3       0.85      0.86      0.85      1358
           4       0.68      0.31      0.42        62
           5       0.66      0.48      0.56      2066
           6       0.88      0.63      0.73      5156
           7       0.02      0.01      0.01        96
           8       0.98      0.99      0.98     21421
           9       0.97      0.96      0.96      6327

    accuracy                           0.92     61699
   macro avg       0.78      0.72      0.74     61699
weighted avg       0.92      0.92      0.92     61699



---

### no preprocess, treshold

In [21]:
## no prep, treshold
clf_mod = svm.LinearSVC(random_state=seed)
clf_mod.fit(X_train_mod, y_train)
y_mod_pred = clf_mod.predict(X_test_mod)

print(accuracy_score(y_test, y_mod_pred))

0.9485396609290414


In [22]:
clf = clf_mod
x = X_val_mod
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     1     0     0     0     0     0     0     0     0]
 [    0 17651   129   111     0   296    53     0     0     9]
 [    0    25  5342     6     0    70     0     0     5     0]
 [    0    97    16  1219     0    13     4     0     9     0]
 [    0     9     1     0    34     2     1     0     0    15]
 [   23   104   168    12    13  1233    21    55   417    20]
 [    0  1104     1     0     0    16  4020     0     0    15]
 [    5     4     0     0     0     7     1    71     0     8]
 [    0    12   139    36     0    31     2     0 21201     0]
 [    0    76     0     0     0     7   108     1     0  6135]]
0.9468711000178285
0.9452896689446899
0.9462217446436155
0.9468711000178285
0.8517908213108194
0.8651984366069223
0.8468591948834774
0.8468591948834774


In [23]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1516
           1       0.92      0.97      0.95     18249
           2       0.92      0.98      0.95      5448
           3       0.88      0.90      0.89      1358
           4       0.72      0.55      0.62        62
           5       0.74      0.60      0.66      2066
           6       0.95      0.78      0.86      5156
           7       0.56      0.74      0.64        96
           8       0.98      0.99      0.98     21421
           9       0.99      0.97      0.98      6327

    accuracy                           0.95     61699
   macro avg       0.87      0.85      0.85     61699
weighted avg       0.95      0.95      0.95     61699



---

### no preprocess, no threshold

In [24]:
## no prep, no treshold
clf = svm.LinearSVC(random_state=seed)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.9485558689098512


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [25]:
clf = clf
x = X_val
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     0     0     0     0     0     1     0     0     0]
 [    0 17649   129   111     0   297    55     0     0     8]
 [    0    25  5342     6     0    70     0     0     5     0]
 [    0    97    16  1220     0    12     4     0     9     0]
 [    0     9     1     0    34     2     1     0     0    15]
 [   24   103   168    12    13  1234    21    53   418    20]
 [    0  1101     1     0     0    17  4022     0     0    15]
 [    5     4     0     0     0     7     1    71     0     8]
 [    1    12   138    36     0    31     2     0 21201     0]
 [    0    76     0     0     0     7   106     1     0  6137]]
0.9469359308902899
0.9453554513969803
0.9462641535164874
0.9469359308902899
0.8524135714220378
0.8659768766127222
0.8470406761093322
0.8470406761093322


In [26]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1516
           1       0.93      0.97      0.95     18249
           2       0.92      0.98      0.95      5448
           3       0.88      0.90      0.89      1358
           4       0.72      0.55      0.62        62
           5       0.74      0.60      0.66      2066
           6       0.95      0.78      0.86      5156
           7       0.57      0.74      0.64        96
           8       0.98      0.99      0.98     21421
           9       0.99      0.97      0.98      6327

    accuracy                           0.95     61699
   macro avg       0.87      0.85      0.85     61699
weighted avg       0.95      0.95      0.95     61699



In [27]:
# from sklearn.metrics import sensitivity_score, false_positive_rate_score, specificity_score, false_negative_rate_score

# print(sensitivity_score(y_val, y_pred))
# print(false_positive_rate_score(y_val, y_pred))
# print(specificity_score(y_val, y_pred))
# print(false_negative_rate_score(y_val, y_pred))

ImportError: cannot import name 'sensitivity_score' from 'sklearn.metrics' (/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/__init__.py)

---

## minmax, threshold

In [14]:
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, balanced_accuracy_score
from sklearn import svm

In [15]:
## minmax, threshold
clf = svm.LinearSVC(random_state=seed)
clf.fit(X_train_mod, y_train)
y_pred = clf.predict(X_test_mod)

print(accuracy_score(y_test, y_pred))

0.8876138610651885


In [16]:
clf = clf
x = X_val_mod
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     0     0     0     0     0     1     0     0     0]
 [    0 16834   139    72     0     9   610     0   583     2]
 [    0    68  5357    11     0    12     0     0     0     0]
 [    0   189     8   545     0     3   528     0    85     0]
 [    1     5     4     0     3     1     0     0     0    48]
 [   38   161   175    13     7   902     7     0   725    38]
 [    0   888     1     9     2     4  2829     0   480   943]
 [   33    15     2     0     0     0     0     0    41     5]
 [    0   153   288   109     0    33    14     0 20824     0]
 [    0   238     8     0     4     3   137     0    22  5915]]
0.8869511661453184
0.8782729136303037
0.8803700457326579
0.8869511661453184
0.645849877213308
0.7047474629468323
0.6247097072845886
0.6247097072845886


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1516
           1       0.91      0.92      0.91     18249
           2       0.90      0.98      0.94      5448
           3       0.72      0.40      0.51      1358
           4       0.19      0.05      0.08        62
           5       0.93      0.44      0.59      2066
           6       0.69      0.55      0.61      5156
           7       0.00      0.00      0.00        96
           8       0.91      0.97      0.94     21421
           9       0.85      0.93      0.89      6327

    accuracy                           0.89     61699
   macro avg       0.70      0.62      0.65     61699
weighted avg       0.88      0.89      0.88     61699



/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

---

## minmax, no threshold

In [18]:
## minmax, no threshold
clf = svm.LinearSVC(random_state=seed)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.8878245648157153


In [19]:
clf = clf
x = X_val
y_pred = clf.predict(x)

print(confusion_matrix(y_val, y_pred))

print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='weighted'))
print(precision_score(y_val, y_pred, average='weighted'))
print(recall_score(y_val, y_pred, average='weighted'))
print(f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))
print(balanced_accuracy_score(y_val, y_pred))

[[ 1515     0     0     0     0     0     1     0     0     0]
 [    0 16833   139    78     0     9   605     0   583     2]
 [    0    68  5357    11     0    12     0     0     0     0]
 [    0   189     8   568     0     3   505     0    85     0]
 [    1     5     4     0     3     1     0     0     0    48]
 [   38   161   175    12     7   902     7     0   725    39]
 [    0   888     1     7     2     4  2827     0   483   944]
 [   33    15     2     0     0     0     0     0    41     5]
 [    0   153   288   110     0    33    13     0 20824     0]
 [    0   237     8     0     4     3   136     0    22  5917]]
0.8873077359438565
0.878722295626932
0.88082867470923
0.8873077359438565
0.6475110522783976
0.7056663132132548
0.6263907154882808
0.6263907154882808


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1516
           1       0.91      0.92      0.91     18249
           2       0.90      0.98      0.94      5448
           3       0.72      0.42      0.53      1358
           4       0.19      0.05      0.08        62
           5       0.93      0.44      0.59      2066
           6       0.69      0.55      0.61      5156
           7       0.00      0.00      0.00        96
           8       0.91      0.97      0.94     21421
           9       0.85      0.94      0.89      6327

    accuracy                           0.89     61699
   macro avg       0.71      0.63      0.65     61699
weighted avg       0.88      0.89      0.88     61699



/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi